In [54]:
import tensorflow as tf
from tensorflow import data
import multiprocessing
import numpy as np
import os
import re
import itertools

In [355]:
#mdl_ha= 'ha_bilstm16_emb8_dr08_lr1m4-25-1m3_bs256_voc15k-swin_v43'
mdl_ha= 'ha_bilstm32_emb8_dr08_lr5m4-25-5m3_bs256_voc15k-swin_v48'
mdl_è = 'è_bilstm16_emb8_dr08_lr5m3-0p3eps-1m3-1eps-5m4_bs256_voc15k-swin_v46'
mdl_hai = 'hai_bilstm32_emb8_dr08_lr5m5-25eps-5m6-50eps-1p4m4_bs256_voc15k-swin_v12'
mdl_ho = 'ho_bilstm16_emb8_dr08_lr1m2-5eps-1m3-15eps-1p4m4_bs256_voc15k-swin_v39'
mdl_sarà = 'sarà_bilstm16_emb8_dr08_lr1m3-3eps-1m3_bs256_voc15k-swin_v9'
#mdl_sei = 'sei_lstm32_emb5_dr08_lr5m3-6eps-1m4-exdec_voc10k-swin_v6'
mdl_sei = 'sei_lstm32_emb5_dr08_lr5m3-6eps-1m4-exdec_voc10k-swin_pb'
#mdl_hanno = 'hanno_bilstm64_emb16_dr08_lr565-25eps-1m7-50eps-5m8_bs256_voc10k_v23'
mdl_hanno = 'hanno_bilstm32_emb16_dr08_lr5m4-25eps-1m3-50eps-5m5_bs256_voc10k_v22'
mdl_i = 'i_bilstm32_emb8_dr08_lr5m4-25eps-5m6-50eps-1p4m4_bs256_voc15k-swin_v6'


In [356]:
def predictor(model_name):
    amb = model_name.split('_')[0]
    model_dir = '../checkpoints/checkpoints_'+amb+'/{}'.format(model_name)
    #model_dir = os.path.join(os.getcwd(),'trained_models_six/{}'.format(model_name))
    export_dir = model_dir + "/export/predict/"
    saved_model_dir= export_dir + "/" + os.listdir(path=export_dir)[-1]  
    return tf.contrib.predictor.from_saved_model(export_dir = saved_model_dir,signature_def_key="prediction")

output = lambda amb, sentence: mdl_amb_dict[amb]({'sentence':sentence})

In [357]:
mdl_amb_dict = {'ha': predictor(mdl_ha), 'a': predictor(mdl_ha), 
                'è': predictor(mdl_è), 'e': predictor(mdl_è),
                'hai': predictor(mdl_hai), 'ai': predictor(mdl_hai),
               'ho': predictor(mdl_ho), 'o': predictor(mdl_ho),
               'hanno': predictor(mdl_hanno), 'anno': predictor(mdl_hanno),
               'sarà': predictor(mdl_sarà), 'sara': predictor(mdl_sarà),
               'sei': predictor(mdl_sei), '6': predictor(mdl_sei)}

amb_list = list(mdl_amb_dict.keys())
print('\n', amb_list)

ha_class_dict = {'0': 'ha', '1': 'a'}
è_class_dict = {'0': 'è', '1': 'e'}
hai_class_dict = {'0': 'hai', '1': 'ai'}
ho_class_dict = {'0': 'ho', '1': 'o'}
hanno_class_dict = {'0': 'hanno', '1': 'anno'}
sarà_class_dict = {'0': 'sarà', '1': 'sara'}
sei_class_dict = {'0': 'sei', '1': '6'}


amb_class_dicts = {'ha': ha_class_dict, 'a': ha_class_dict, 'è': è_class_dict, 'e': è_class_dict,
                  'hai': hai_class_dict, 'ai': hai_class_dict, 'ho': ho_class_dict, 'o': ho_class_dict,
                  'hanno': hanno_class_dict, 'anno': hanno_class_dict, 'sarà': sarà_class_dict, 'sara': sarà_class_dict,
                  'sei': sei_class_dict, '6': sei_class_dict}

amb_siblings_dict={'ha': 'a', 'è': 'e', 'hai': 'ai', 'ho': 'o', 'hanno': 'anno', 'sarà': 'sara', 'sei':'6'}

output = lambda amb, sentence: mdl_amb_dict[amb]({'sentence':sentence})

INFO:tensorflow:Restoring parameters from b'../checkpoints/checkpoints_ha/ha_bilstm32_emb8_dr08_lr5m4-25-5m3_bs256_voc15k-swin_v48/export/predict//1585829406/variables/variables'
INFO:tensorflow:Restoring parameters from b'../checkpoints/checkpoints_ha/ha_bilstm32_emb8_dr08_lr5m4-25-5m3_bs256_voc15k-swin_v48/export/predict//1585829406/variables/variables'
INFO:tensorflow:Restoring parameters from b'../checkpoints/checkpoints_\xc3\xa8/\xc3\xa8_bilstm16_emb8_dr08_lr5m3-0p3eps-1m3-1eps-5m4_bs256_voc15k-swin_v46/export/predict//1585663372/variables/variables'
INFO:tensorflow:Restoring parameters from b'../checkpoints/checkpoints_\xc3\xa8/\xc3\xa8_bilstm16_emb8_dr08_lr5m3-0p3eps-1m3-1eps-5m4_bs256_voc15k-swin_v46/export/predict//1585663372/variables/variables'
INFO:tensorflow:Restoring parameters from b'../checkpoints/checkpoints_hai/hai_bilstm32_emb8_dr08_lr5m5-25eps-5m6-50eps-1p4m4_bs256_voc15k-swin_v12/export/predict//1585748009/variables/variables'
INFO:tensorflow:Restoring parameters f

In [358]:
def transform(sentence, PRINT=False):
    s = str(sentence)
    s = s.rstrip().lower()
    probs = []
    if PRINT: print(s) 
    for amb in amb_list:
        if amb in s.split(' '):
            s = re.sub(r"\b%s\b" %(amb),"tannutuva", s)
            if PRINT: print(s, '    ', amb)
            prob = np.around(output(amb, [s])['probabilities'][0:], decimals=3)
            probs.append(list(prob[0]))
            if prob[0,0] > 0.5:
                label='0' 
                s = re.sub(r"\btannutuva\b",amb_class_dicts[amb][label], s)
                if PRINT: print(s, '    ', label)
            else: 
                label='1' 
                s = re.sub(r"\btannutuva\b",amb_class_dicts[amb][label], s)
                if PRINT: print(s, '    ', label)
        else:
            if PRINT: print(s)
            continue
    return s, probs


            
transform('a tre anni e mezzo ho quattro')           
transform('il meteo oggi a massa') 
transform('che tempo fa oggi a l aquila') 
transform('che tempo fa oggi a trenta') 
transform('in che anno siamo') 

('in che anno siamo', [[0.099, 0.901]])

In [362]:
transform('che tempo fa oggi a soiano e domani o ieri') 
transform('hai degli impegni oggi o nel weekend e cosa vorresti fare') 
transform('hai visto al cinema o a casa il film di woody allen') 
transform('cosa è che vogliamo fare stasera o domani') 
transform('aggiungi hai preferiti della rubrica il contatto visa') 



transform('di che anno è la canzone torero camillo') 
transform('che tempo oggi a reggio emilia') 

('che tempo oggi a reggio emilia', [[0.009, 0.991]])

In [363]:
def test_on_tickets(file,):
    
    file  = [ line.rstrip().lower() for line in file]
    file  = list(file)
    tot_sents = len(file)
    tot_amb = 0
    
    cnt_wrg = 0
    tot_amb = 0
    # boolean that is True if postagging is active (ex model name: è_pos_blablabla)
    for s in file:
        transformed = transform(str(s))
        if transformed[0] != s:
            cnt_wrg += 1
            print('tr :', transformed)
            
        
    #print('--------------------', cnt_wrg, tot_amb, cnt_wrg/float(tot_amb)*100.0)

In [364]:
#TicketsTest = open("../tickets_test/change_v1.txt", "r")
#TicketsTest = open("../tickets_test/regression_v1.txt", "r")
TicketsTest = open("../tickets_test/NeuralTaggerTicketsTest.txt", "r")
LoadTest = open("../tickets_test/NeuralTaggerLoadTest.txt", "r")
NRTest = open("../tickets_test/NeuralTaggerNonRegressionTicketsTest.txt", "r")
ASRGiulio = open("../tickets_test/NeuralTaggerASRfailureGiulio.txt", "r")
UMSHanno = open("../tickets_test/NeuralTaggerTicketsTest_hanno.txt", "r")



testlist = [TicketsTest, LoadTest, NRTest, ASRGiulio, UMSHanno ]
print('---------------\n')
for test in testlist:
    test_on_tickets(test)
    print('---------------\n')

---------------

tr : ('nove 6 due otto da roma termini è in orario', [[0.966, 0.034], [0.0, 1.0]])
tr : ('invieresti un sms per me il destinatario è marta è il messaggio è di a saverio che lo chiamo più tardi o gli scrivo qualcosa', [[0.735, 0.265], [0.996, 0.004], [0.011, 0.989], [0.0, 1.0]])
tr : ('che ora è è che giorno è oggi', [[0.734, 0.266], [0.943, 0.057]])
tr : ('che giorno è oggi è che ore sono', [[0.843, 0.157], [0.794, 0.206]])
tr : ('che giorno e e che ore sono oggi', [[0.171, 0.829], [0.325, 0.675]])
tr : ('che giorno è che ora è oggi', [[0.775, 0.225], [0.92, 0.08]])
tr : ('nove 6 due otto da roma termini è in orario', [[0.966, 0.034], [0.0, 1.0]])
tr : ('è a gennaio', [[0.991, 0.009], [0.991, 0.009], [0.101, 0.899]])
tr : ('è a febbraio', [[0.996, 0.004], [0.996, 0.004], [0.097, 0.903]])
tr : ('è a marzo', [[0.993, 0.007], [0.993, 0.007], [0.138, 0.862]])
tr : ('è a maggio', [[0.993, 0.007], [0.993, 0.007], [0.06, 0.94]])
tr : ('è a giugno', [[0.994, 0.006], [0.994, 0.

In [169]:
transform('in che anno costruito la tour eiffel') 
transform('ho l infezione coronavirus') 
transform('ho l infezione del virus corona') 
transform('hai quiz pasquali') 
transform('se pensi che questo abbia un lieto fine non hai prestato attenzione') 

transform('che tempo oggi a reggio emilia')

('che tempo oggi ha reggio emilia', [[0.691, 0.309], [0.691, 0.309]])

In [ ]:
imposta a uno il bilanciamento del colore dello schermo
che tempo oggi a reggio emilia
il tempo a ortonovo
che ha fatto la serie a 

### i_e case 

In [206]:
mdl_amb_dict = {'i': predictor(mdl_i), 'e': predictor(mdl_i)}

amb_list = list(mdl_amb_dict.keys())
print('\n', amb_list)

i_class_dict = {'0': 'e', '1': 'i'}



amb_class_dicts = { 'i': i_class_dict, 'e': i_class_dict}

amb_siblings_dict={'i':'e'}

output = lambda amb, sentence: mdl_amb_dict[amb]({'sentence':sentence})

INFO:tensorflow:Restoring parameters from b'../checkpoints/checkpoints_i/i_bilstm32_emb8_dr08_lr5m4-25eps-5m6-50eps-1p4m4_bs256_voc15k-swin_v6/export/predict//1585757135/variables/variables'
INFO:tensorflow:Restoring parameters from b'../checkpoints/checkpoints_i/i_bilstm32_emb8_dr08_lr5m4-25eps-5m6-50eps-1p4m4_bs256_voc15k-swin_v6/export/predict//1585757135/variables/variables'

 ['i', 'e']


In [225]:
def transform(sentence, PRINT=False):
    s = str(sentence)
    s = s.rstrip().lower()
    probs = []
    if PRINT: print(s) 
    for amb in amb_list:
        if amb in s.split(' '):
            s = re.sub(r"\b%s\b" %(amb),"tannutuva", s)
            if PRINT: print(s, '    ', amb)
            prob = np.around(output(amb, [s])['probabilities'][0:], decimals=3)
            probs.append(list(prob[0]))
            if prob[0,0] > 0.5:
                label='0' 
                s = re.sub(r"\btannutuva\b",amb_class_dicts[amb][label], s)
                if PRINT: print(s, '    ', label)
            else: 
                label='1' 
                s = re.sub(r"\btannutuva\b",amb_class_dicts[amb][label], s)
                if PRINT: print(s, '    ', label)
        else:
            if PRINT: print(s)
            continue
    return s, probs


def transform_ei(sentence, trigger=0.98, PRINT=False):
    s = str(sentence)
    s = s.rstrip().lower()
    probs = []
    for amb in ['i', 'e']:
        if amb in s.split(' '):
            if PRINT: print(s) 
            s = re.sub(r"\b%s\b" %(amb),"tannutuva", s)
            if PRINT: print(s, '    ', amb)
            prob = np.around(output('i', [s])['probabilities'][0:], decimals=3)
            probs.append(list(prob[0]))
            # change tannutuva to "e" only if you are above trigger (say 97%) sure!
            if prob[0,0] > trigger:
                label='0' 
                s = re.sub(r"\btannutuva\b",'e', s)
                if PRINT: print(s, '    ', label)
            else: 
                label='1' 
                s = re.sub(r"\btannutuva\b",'i', s)
                if PRINT: print(s, '    ', label)
        else:
            continue
    return s, probs

#tu i siri siete amiche
#screenshot i invia
#qual è la distanza tra giove i marte

In [227]:
transform_ei('qual è la distanza tra giove i marte', trigger=0.90, PRINT=True) 


qual è la distanza tra giove i marte
qual è la distanza tra giove tannutuva marte      i
qual è la distanza tra giove e marte      0
qual è la distanza tra giove e marte
qual è la distanza tra giove tannutuva marte      e
qual è la distanza tra giove e marte      0


('qual è la distanza tra giove e marte', [[0.959, 0.041], [0.959, 0.041]])

In [234]:

def test_on_tickets_ei(file,):
    file  = [ line.rstrip().lower() for line in file]
    file  = list(file)
    tot_sents = len(file)
    tot_amb = 0
    
    cnt_wrg = 0
    tot_amb = 0
    # boolean that is True if postagging is active (ex model name: è_pos_blablabla)
    for s in file:
        transformed = transform_ei(str(s),trigger=0.90)
        if transformed[0] != s:
            cnt_wrg += 1
            print('tr :', transformed)
    #print('--------------------', cnt_wrg, tot_amb, cnt_wrg/float(tot_amb)*100.0)

In [235]:
TicketsTest = open("../tickets_test/NeuralTaggerTicketsTest.txt", "r")
LoadTest = open("../tickets_test/NeuralTaggerLoadTest.txt", "r")
NRTest = open("../tickets_test/NeuralTaggerNonRegressionTicketsTest.txt", "r")
ASRGiulio = open("../tickets_test/NeuralTaggerASRfailureGiulio.txt", "r")
UMSHanno = open("../tickets_test/NeuralTaggerTicketsTest_hanno.txt", "r")


testlist = [TicketsTest, LoadTest, NRTest, ASRGiulio, UMSHanno ]
print('---------------\n')
for test in testlist:
    test_on_tickets_ei(test)
    print('---------------\n')

---------------

tr : ('scansiona i traduci questo testo', [[0.846, 0.154]])
tr : ('sveglia tra un ora i quaranta', [[0.835, 0.165]])
tr : ('scansiona i traduci questo testo', [[0.846, 0.154]])
tr : ('sveglia tra un ora i quaranta', [[0.835, 0.165]])
tr : ('quando giocano napoli i juventus', [[0.728, 0.272]])
tr : ('ora i data', [[0.68, 0.32]])
tr : ('quando giocano napoli i juventus', [[0.728, 0.272]])
tr : ('ora i data', [[0.68, 0.32]])
tr : ('che mese è pasqua nel duemila i venti', [[0.631, 0.369]])
tr : ('che ore sono i che giorno è', [[0.791, 0.209]])
tr : ('che ore sono i che giorno è oggi', [[0.807, 0.193]])
tr : ('che ora è i che giorno è oggi', [[0.002, 0.998]])
tr : ('che giorno è i che ore sono oggi', [[0.031, 0.969]])
tr : ('metti lo zoom della camera a dici per i scatta una foto', [[0.012, 0.988]])
tr : ('scansiona i traduci questo testo', [[0.846, 0.154]])
tr : ('sveglia tra un ora i quaranta', [[0.835, 0.165]])
tr : ('scansiona i traduci questo testo', [[0.846, 0.154]])
